Using baseline model i was not able to saturate my gpu, here i will try to asses if the loader was the bottleneck 

# Preparations

In [1]:
import sys
sys.path.append('../src')

import torch
from torch import nn

import torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
from torchinfo import summary
from tqdm.auto import tqdm
from timeit import default_timer as timer


from helper_functions import train_step, test_step, accuracy_fn, print_train_time

device = 'cuda' if torch.cuda.is_available() else 'cpu'

/home/aaa/Desktop/deep_learning/ConvolutionalNeuralNeutworks/venv_deep_learning/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
seed = 213
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

In [3]:
class BaselineModel(nn.Module):
    """
    Model architecture copying TinyVGG from: 
    https://poloclub.github.io/cnn-explainer/
    """
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int):
        super().__init__()
        self.block_1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape, 
                      out_channels=hidden_units, 
                      kernel_size=3,
                      stride=1,
                      padding=1), 
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units, 
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,
                         stride=2) 
        )
        self.block_2 = nn.Sequential(
            nn.Conv2d(hidden_units, hidden_units, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(hidden_units, hidden_units, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=hidden_units*8*8, 
                      out_features=output_shape)
        )
    
    def forward(self, x: torch.Tensor):
        x = self.block_1(x)
        x = self.block_2(x)
        x = self.classifier(x)
        return x

In [4]:
def run_model(cinic_train, cinic_test, model, loss_fn, optimizer, epochs=3):
    train_time_start_model_2 = timer()

    # Train and test model 
    for epoch in tqdm(range(epochs)):
        print(f"Epoch: {epoch}\n---------")
        train_step(data_loader=cinic_train, 
            model=model, 
            loss_fn=loss_fn,
            optimizer=optimizer,
            accuracy_fn=accuracy_fn,
            device=device
        )
        test_step(data_loader=cinic_test,
            model=model,
            loss_fn=loss_fn,
            accuracy_fn=accuracy_fn,
            device=device
        )

    train_time_end_model_2 = timer()
    total_train_time_model_2 = print_train_time(start=train_time_start_model_2,
                                            end=train_time_end_model_2,
                                            device=device)

# Experiments

### the one i used originally

In [5]:
cinic_directory = '../data'
cinic_train = torch.utils.data.DataLoader(
    torchvision.datasets.ImageFolder(cinic_directory + '/train',
    	transform=transforms.ToTensor()),
    batch_size=64, shuffle=True)

cinic_test = torch.utils.data.DataLoader(
    torchvision.datasets.ImageFolder(cinic_directory + '/test',
    	transform=transforms.ToTensor()),
    batch_size=64, shuffle=True)

In [7]:
model = BaselineModel(input_shape=3, 
    hidden_units=10, 
    output_shape=10).to(device)



loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

run_model(cinic_train, cinic_test, model, loss_fn, optimizer, epochs=3)

  0%|          | 0/3 [00:00<?, ?it/s]

Epoch: 0
---------
Train loss: 1.97782 | Train accuracy: 26.68%


 33%|███▎      | 1/3 [00:19<00:39, 19.56s/it]

Test loss: 1.82904 | Test accuracy: 31.21%

Epoch: 1
---------
Train loss: 1.82565 | Train accuracy: 31.31%


 67%|██████▋   | 2/3 [00:39<00:19, 19.53s/it]

Test loss: 1.82044 | Test accuracy: 31.40%

Epoch: 2
---------
Train loss: 1.77338 | Train accuracy: 33.80%


100%|██████████| 3/3 [00:58<00:00, 19.53s/it]

Test loss: 1.77385 | Test accuracy: 33.20%


Train time on cuda: 58.578 seconds


### two workers

In [8]:
cinic_directory = '../data'
cinic_train = torch.utils.data.DataLoader(
    torchvision.datasets.ImageFolder(cinic_directory + '/train',
    	transform=transforms.ToTensor()),
    batch_size=64, shuffle=True, num_workers=2)

cinic_test = torch.utils.data.DataLoader(
    torchvision.datasets.ImageFolder(cinic_directory + '/test',
    	transform=transforms.ToTensor()),
    batch_size=64, shuffle=True, num_workers=2)

In [9]:
model = BaselineModel(input_shape=3, 
    hidden_units=10, 
    output_shape=10).to(device)



loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

run_model(cinic_train, cinic_test, model, loss_fn, optimizer, epochs=3)

  0%|          | 0/3 [00:00<?, ?it/s]

Epoch: 0
---------
Train loss: 1.93246 | Train accuracy: 27.86%


 33%|███▎      | 1/3 [00:10<00:20, 10.42s/it]

Test loss: 1.79026 | Test accuracy: 32.72%

Epoch: 1
---------
Train loss: 1.76192 | Train accuracy: 33.96%


 67%|██████▋   | 2/3 [00:20<00:10, 10.29s/it]

Test loss: 1.74561 | Test accuracy: 33.98%

Epoch: 2
---------
Train loss: 1.73714 | Train accuracy: 35.16%


100%|██████████| 3/3 [00:30<00:00, 10.29s/it]

Test loss: 1.72203 | Test accuracy: 36.65%


Train time on cuda: 30.856 seconds


### 6 workers

In [20]:
cinic_directory = '../data'
cinic_train = torch.utils.data.DataLoader(
    torchvision.datasets.ImageFolder(cinic_directory + '/train',
    	transform=transforms.ToTensor()),
    batch_size=64, shuffle=True, num_workers=6)

cinic_test = torch.utils.data.DataLoader(
    torchvision.datasets.ImageFolder(cinic_directory + '/test',
    	transform=transforms.ToTensor()),
    batch_size=64, shuffle=True, num_workers=6)

In [21]:
model = BaselineModel(input_shape=3, 
    hidden_units=10, 
    output_shape=10).to(device)



loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

run_model(cinic_train, cinic_test, model, loss_fn, optimizer, epochs=3)

  0%|          | 0/3 [00:00<?, ?it/s]

Epoch: 0
---------
Train loss: 2.30366 | Train accuracy: 10.01%


 33%|███▎      | 1/3 [00:04<00:09,  4.98s/it]

Test loss: 2.30333 | Test accuracy: 10.00%

Epoch: 1
---------
Train loss: 2.30355 | Train accuracy: 10.01%


 67%|██████▋   | 2/3 [00:09<00:04,  4.94s/it]

Test loss: 2.30386 | Test accuracy: 10.00%

Epoch: 2
---------
Train loss: 2.30366 | Train accuracy: 9.73%


100%|██████████| 3/3 [00:14<00:00,  4.93s/it]

Test loss: 2.30302 | Test accuracy: 9.99%


Train time on cuda: 14.797 seconds


### two workers and pin_memory

In [22]:
cinic_directory = '../data'
cinic_train = torch.utils.data.DataLoader(
    torchvision.datasets.ImageFolder(cinic_directory + '/train',
    	transform=transforms.ToTensor()),
    batch_size=64, shuffle=True, num_workers=2, pin_memory=True)

cinic_test = torch.utils.data.DataLoader(
    torchvision.datasets.ImageFolder(cinic_directory + '/test',
    	transform=transforms.ToTensor()),
    batch_size=64, shuffle=True, num_workers=2, pin_memory=True)

In [19]:
model = BaselineModel(input_shape=3, 
    hidden_units=10, 
    output_shape=10).to(device)



loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

run_model(cinic_train, cinic_test, model, loss_fn, optimizer, epochs=3)

  0%|          | 0/3 [00:00<?, ?it/s]

Epoch: 0
---------
Train loss: 2.30347 | Train accuracy: 10.07%


 33%|███▎      | 1/3 [00:10<00:20, 10.44s/it]

Test loss: 2.30368 | Test accuracy: 10.00%

Epoch: 1
---------
Train loss: 2.30349 | Train accuracy: 10.12%


 67%|██████▋   | 2/3 [00:20<00:10, 10.39s/it]

Test loss: 2.30307 | Test accuracy: 10.00%

Epoch: 2
---------
Train loss: 2.30358 | Train accuracy: 9.84%


100%|██████████| 3/3 [00:31<00:00, 10.41s/it]

Test loss: 2.30490 | Test accuracy: 10.00%


Train time on cuda: 31.228 seconds


### two workers and drop_last

In [23]:
cinic_directory = '../data'
cinic_train = torch.utils.data.DataLoader(
    torchvision.datasets.ImageFolder(cinic_directory + '/train',
    	transform=transforms.ToTensor()),
    batch_size=64, shuffle=True, num_workers=2, drop_last=True)

cinic_test = torch.utils.data.DataLoader(
    torchvision.datasets.ImageFolder(cinic_directory + '/test',
    	transform=transforms.ToTensor()),
    batch_size=64, shuffle=True, num_workers=2, drop_last=True)

In [24]:
model = BaselineModel(input_shape=3, 
    hidden_units=10, 
    output_shape=10).to(device)



loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

run_model(cinic_train, cinic_test, model, loss_fn, optimizer, epochs=3)

  0%|          | 0/3 [00:00<?, ?it/s]

Epoch: 0
---------
Train loss: 2.30359 | Train accuracy: 10.04%


 33%|███▎      | 1/3 [00:10<00:20, 10.25s/it]

Test loss: 2.30341 | Test accuracy: 10.00%

Epoch: 1
---------
Train loss: 2.30354 | Train accuracy: 9.95%


 67%|██████▋   | 2/3 [00:20<00:10, 10.23s/it]

Test loss: 2.30310 | Test accuracy: 10.00%

Epoch: 2
---------
Train loss: 2.30360 | Train accuracy: 10.00%


100%|██████████| 3/3 [00:30<00:00, 10.24s/it]

Test loss: 2.30309 | Test accuracy: 10.00%


Train time on cuda: 30.734 seconds


### Again just two workers to make sure it works 

In [27]:
cinic_directory = '../data'
cinic_train = torch.utils.data.DataLoader(
    torchvision.datasets.ImageFolder(cinic_directory + '/train',
    	transform=transforms.ToTensor()),
    batch_size=64, shuffle=True, num_workers=2)

cinic_test = torch.utils.data.DataLoader(
    torchvision.datasets.ImageFolder(cinic_directory + '/test',
    	transform=transforms.ToTensor()),
    batch_size=64, shuffle=True, num_workers=2)

In [ ]:
model = BaselineModel(input_shape=3, 
    hidden_units=10, 
    output_shape=10).to(device)



loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

run_model(cinic_train, cinic_test, model, loss_fn, optimizer, epochs=3)

  0%|          | 0/3 [00:00<?, ?it/s]

Epoch: 0
---------
Train loss: 1.94991 | Train accuracy: 27.40%


 33%|███▎      | 1/3 [00:10<00:20, 10.34s/it]

Test loss: 1.78910 | Test accuracy: 33.94%

Epoch: 1
---------
Train loss: 1.77346 | Train accuracy: 34.10%


 67%|██████▋   | 2/3 [00:20<00:10, 10.26s/it]

Test loss: 1.74556 | Test accuracy: 35.35%

Epoch: 2
---------
